In [ ]:
pip install jointtsmodel

In [1]:
import os
cwd = os.getcwd()
cwd

'C:\\Users\\User\\jh2\\JST'

### 1. 감성사전 설정

In [39]:
# KNU 감성사전 기반 lexicon 생성 (긍정단어 - 1, 2점 -> 1점 // 부정단어 - -1, -2점 -> -1점 // 중립단어 제외)

import json
import pandas as pd

sentiment_df = pd.DataFrame(index=['Word','Sentiment'])
word_list = []
sentiment_list = []

with open('SentiWord_info.json', encoding='utf-8') as f:
    
    data = json.load(f)
    
    for i in data:
        root = i['word_root']
        score = int(i['polarity'])
        
        if score >= 1:
            score = 1
        
        elif score <= -1:
            score = -1
        
        elif score == 0:
            continue
        
        out = pd.DataFrame({'Word':[root], 'Sentiment':[score]})
        sentiment_df = pd.concat([sentiment_df, out])
    
    
sentiment_df = sentiment_df.reset_index(drop=True).dropna()
sentiment_df.to_excel('prior_sentiment_kor.xlsx', encoding='utf-8-sig', index=False)

C:\Users\User\anaconda3\envs\jh2\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


### 2.전처리함수 설정

In [137]:
import re
import pandas as pd
import openpyxl
import os
import nltk

#visualization
import matplotlib as mpl
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

from konlpy.tag import Mecab, Kkma, Okt
from iteration_utilities import unique_everseen
from wordcloud import WordCloud
from matplotlib import font_manager, rc
from bs4 import BeautifulSoup



path_list = ['C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/블로그/2020_2021/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/유튜브/2020_2021/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/인스타/2020_2021/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/트위터/2020_2021/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/블로그/2022',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/유튜브/2022/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/인스타/2022',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/전처리 이후/트위터/2022/',
             'C:/Users/User/Desktop/ESG_자료/esg_텍스트/분석용/보고서/']


def file_process(n, dt='xlsx'):
    # 경로설정
    path = path_list[n]

    # 결과 추출을 위한 이름 설정
    name = set_name(path)

    df = open_file(path, datatype=dt)

    tqdm.pandas(desc="Tokenization Progress")
    df['tokens'] = df['text'].progress_apply(preprocess, custom_dict=custom_dict)
    df['text'] = df['text'].progress_apply(pre_process)


    # 중복되는 df 제거
    df.drop_duplicates(subset=['text'], keep='first', inplace=True)
    df['num_tokens'] = df['tokens'].map(len)

    # 토큰값이 0인 행 제거
    df = df.loc[df['num_tokens'] != 0]

    # index 초기화
    df.reset_index(drop=True, inplace=True)
    
    return df

"""

파일 불러오기를 위한 함수 설정 

- path: 경로
- datatype= 'docx' or 'xlsx'
- min_text_docx = n 문단최소글자수 (default == 5)
- min_text_xlsx = n 엑셀 최소 글자수 (default == 5)

"""

def open_file(path, datatype=None, min_text_docx = 5, min_text_xlsx = 5):
    pd.set_option('display.max_colwidth', None)
    files_xlsx = []
    files_docx = []

    if datatype == 'docx':
        files_docx = [file for file in os.listdir(path) if file.endswith('.docx')]
        paragraphs = []
        titles = []
        
        for i in range(len(files_docx)):
            file_name = path+files_docx[i]
            text = docx2txt.process(file_name)
            doc_paragraphs = [p.strip() for p in text.split('\n'*3) if len(p.strip()) > min_text_docx]
            # doc_paragraphs = [p.strip() for p in re.split('\n+', text) if len(p.strip()) > min_text_docx]            
            
            titles += [files_docx[i]] * len(doc_paragraphs)
            paragraphs += doc_paragraphs
            
        df = pd.DataFrame(paragraphs, columns=['text'])
        # df = pd.DataFrame(list(zip(titles, paragraphs)), columns=['title', 'text'])

    elif datatype == 'xlsx':
        files_xlsx = [file for file in os.listdir(path) if file.endswith('.xlsx')]
        df = pd.DataFrame()
        
        if '유튜브' in path:
            for f in files_xlsx:
                file_path = os.path.join(path, f)
                
                df_title = pd.read_excel(file_path, usecols=['제목'])
                df_title = df_title.rename(columns={'제목': 'text'})   
                df_title = df_title.drop_duplicates(keep='first')

                df_text = pd.read_excel(file_path, usecols=['댓글'])
                df_text = df_text.rename(columns={'댓글': 'text'})   
                
                df = pd.concat([df, df_title, df_text], ignore_index=True)
                
#         elif '인스타' in path:
#             for f in files_xlsx:
#                 file_path = os.path.join(path, f)
                
#                 df_text = pd.read_excel(file_path, usecols=['본문'])
#                 df_text = df_text.rename(columns={'본문': 'text'})   
#                 df_text = df_text.drop_duplicates(keep='first')

#                 df_com = pd.read_excel(file_path, usecols=['댓글'])
#                 df_com = df_com.rename(columns={'댓글': 'text'})  
                
                
#                 df = pd.concat([df, df_text, df_com], ignore_index=True)

        # 댓글 각자 문서로 처리
    
        elif '인스타' in path:
            for f in files_xlsx:
                file_path = os.path.join(path, f)
                df_text = pd.read_excel(file_path, usecols=['본문'])
                df_text = df_text.rename(columns={'본문': 'text'})
                df_text = df_text.drop_duplicates(keep='first')
                df_com = pd.read_excel(file_path, usecols=['댓글'])
                df_com = df_com.rename(columns={'댓글': 'text'})
                
                # 각 댓글을 하나의 문서로 분리
                df_com['text'] = df_com['text'].str.split('/')
                df_com = df_com.explode('text')
                
                df = pd.concat([df, df_text, df_com], ignore_index=True)

        elif '트위터' in path:
            for f in files_xlsx:
                file_path = os.path.join(path, f)
                df_read = pd.read_excel(file_path, usecols=['text'])
                df = pd.concat([df, df_read])

        elif '블로그' in path:
            
            for f in files_xlsx:
                file_path = os.path.join(path, f)
                
                # df_title = pd.read_excel(file_path, usecols=['title'])
                # df_title = df_title.rename(columns={'title': 'text'})   
                # df_title = df_title.drop_duplicates(keep='first')

                df_text = pd.read_excel(file_path, usecols=['text'])
                df = pd.concat([df, df_text], ignore_index=True)

                # df = pd.concat([df, df_title, df_text], ignore_index=True)
                

    return df.dropna()

# 형태소 분석을 위한 객체 생성
mecab = Mecab(dicpath='c:/mecab/mecab-ko-dic')

# 1차 전처리 (한글 이외에 모든 텍스트 제거)
def pre_process(text):
    
    pr_text = re.sub(r"[^ㄱ-ㅣ가-힣\s]+|[ㄱ-ㅎㅏ-ㅣ]+", "", text)
    pr_text = re.sub('\\<br\\>',' ',pr_text)
    pr_text = re.sub(r'\s+', ' ', pr_text, flags=re.I)
    pr_text = pr_text.replace("\n", "")
    pr_text = pr_text.strip()
    return pr_text

# 2차 전처리 mecab을 통한 형태소 분석
def analyzer_pos(text, custom_dict):
    tokens = mecab.pos(text)
    # tokens = [ s for s, t in tokens if t in ['XR', 'VV', 'VA', 'NNG', 'NNP'] and len(s) > 1 and s not in stop_word] 
    # tokens = [ s for s, t in tokens if len(s) > 1 and s not in stop_word] # 전체 품사 중 3글자 이상 + 불용어를 제외함
    tokens = [ s for s, t in tokens if t in ['VV', 'VA', 'NNG', 'NNP'] and len(s) > 1 and s not in stop_word] # 동사, 형용사, 명사 중 2글자 이상 + 불용어를 제외함

    # 원하는 단어가 끊여져서 나올경우(우선순위 설정 이후) 합치기
    i = 0
    new_tokens = []

    while i < len(tokens):
        if i == len(tokens) - 1:
            new_tokens.append(tokens[i])
            break
        current_token = tokens[i]
        next_token = tokens[i+1]
        if current_token+next_token in custom_dict:
            new_tokens.append(current_token+next_token)         
            i += 2
        else:
            new_tokens.append(current_token)
            i += 1
    return new_tokens


def set_name(path):
    name = ''
    
    if '보고서' in path:
        name = '보고서'

    elif '인스타' in path:
        name = '인스타'

    elif '블로그' in path:
        name = '블로그'

    elif '트위터' in path:
        name = '트위터'

    elif '유튜브' in path:
        name = '유튜브'
    
    return name


"""
텍스트를 입력받아 사용자 정의 사전을 기반으로 정규표현식 기반 전처리, mecab 형태소 분석을 진행

"""

def preprocess(text, custom_dict=None):
    
    # 정규표현식 처리
    text = pre_process(text)
    
    # mecab 전처리
    tokens = analyzer_pos(text, custom_dict)
    # tokens = ", ".join(tokens)

    return tokens

### 3. 불용어사전 호출

In [150]:
# 불용어
# 길호현. (2018). 텍스트마이닝을 위한 한국어 불용어 목록 연구. 우리말글, 78, 1-25.

with open("stop.txt", "r", encoding="utf8") as f:
    stop_word = []
    stop_text = [stop_word.extend(line.strip().split(", ")) for line in f]


# 개인 전처리를 위한 stop_word 설정
stop_word_jh = ['최태원', '디스클로저', '외신', '출처', '한국', '뉴스', '최초', '공개', '단독', '회사', '상장', '보도', '글로벌', '발행', '래퍼', '출연', '리뷰', '중앙일보', '기자', '이낙연',
                '디렉', '명상', '회장', '신문', '나라', '블로그', '코리아', '미국', '중국', '관련', '대한민국', '대리석', '타임스', '무디스', '법무법인', '변호사', '교수', '기업',
                '타임즈', '인더스', '트리', '습니다', '에서', '속도', '한다', '이승기', '방위대', '일본', '는데', '으로', '처럼', '호남', '야구', '위한', '면서', '지만', '에게', '천억', '투데이', '해요', '펭귄', '팝콘',
                '북마크', '네이버', '네슬레', '자칭', '네요', '코엑스', '노스페이스', '홈페이지', '어야', '전라', '전북', '매일신문', '감사', '영상', '만들', '방송', '슈카', '오늘', '배우', '세상', '사람',
                '다니', '애플', '나오', '삼전', '민주당', '선생', '선수', '이번', '마찬가지', '그동안', '동원', '참치', '맛있', '한화', '그룹', '문재인', '래원', '노래', '좌파', '휠라', '기판',
                '하이닉스', '한미', '입니다', '게임', '넷마블', '삼성', '시멘트', '조선', '백화점', '약품', '의약품', '해양', '효성', '렌탈', '울산', '자동차', '그램', '정답',
                '빅이슈', '삼양', '두부', '부면', '포스코', '하림', '비빔면', '상공', '도시락', '스타', '미주', '라무', '유한', '킴벌리', '농협', '모델', '프로', '아거', '연지', '김병만',
                '떨채', '리비', '프로', '퀴즈', '경기주택공사', '경기', '주택', '공사', '진분', '테슬라', '교보생명', '푸르', '서울', '공단', '삼성전자', '부문', '카카오', '데일리', '임팩트',
                '국내', '시사', '센터', '앨범', '지디', '엔터', '부산', '연구원', '이벤트', '출장', '박스', '안치용', '슈가버블', '배민', '산불', '피해', '클릭', '지수', '하이브', '호비', '경기도',
                '래리', '핑크', '기사', '링크', '윤석열', '생각', '케어', '오전', '오후', '제주', '가능', '내용', '경우', '정도', '소개', '대표', '언서', '학교', '대학교', '학생', '사진', '커피',
                '피부', '생일', '탄생', '월드', '친구', '모닝', '생일']

stop_word.extend(stop_word_jh)
print(stop_word)
print(len(stop_word))

custom_dict = ['지속가능', '이해관계자', '탄소중립', '신재생에너지', '재생에너지', '탄소중립', '탄소제로', '메타버스', '지배구조', '지속가능보고서', '지속가능경영보고서', '사외이사',
               '이상기후', '기후변화', '게임체인저', '동반성장', '고부가가치', '협력사', '가치사슬', '밸류체인', '유연근무', '정보보호', '이행원칙', '노사협의회', '위드코로나', '제로웨이스트']

['가량', '가지', '각', '간', '갖은', '개', '개국', '개년', '개소', '개월', '걔', '거', '거기', '거리', '건', '것', '겨를', '격,', '겸', '고', '군', '군데', '권', '그', '그거', '그것', '그곳', '그까짓', '그네', '그녀', '그놈', '그대', '그래', '그래도,', '그서', '그러나', '그러니', '그러니까', '그러다가', '그러면', '그러면서', '그러므로', '그러자', '그런', '그런', '데', '그럼', '그렇지만', '그루', '그리고', '그리하여', '그분', '그이', '그쪽', '근', '근데', '글쎄', '글쎄요', '기,', '김', '나', '나름', '나위', '남짓', '내', '냥', '너', '너희', '네', '네놈', '녀석', '년', '년대', '년도', '놈', '누구', '니,', '다른', '다만', '단', '달', '달러', '당신', '대', '대로', '더구나', '더욱이', '데', '도', '동', '되', '두', '두세', '두어,', '둥', '듯', '듯이', '등', '등등', '등지', '따라서', '따름', '따위', '딴', '때문', '또', '또는', '또한', '리', '마당', '마련,', '마리', '만', '만큼', '말', '매', '맨', '명', '몇', '몇몇', '모', '모금', '모든', '무렵', '무슨', '무엇', '뭐', '뭣', '미터,', '및', '바', '바람', '바퀴', '박', '발', '발짝', '번', '벌', '법', '별', '본', '부', '분', '뻔', '뿐', '살', '새', '서너', '석', '설,', '섬', '세', '세기', '셈', '쇤네', '수', '순', '스무', '승', '시', '시간', '식', '씨', '아', '아냐', '아니', '아니야', '아무,', '아무개', '아무런', '아아', '아이', '아이고', 

### 4. 데이터 호출 및 전처리

In [151]:
import pandas as pd
import os
import docx2txt
from tqdm import tqdm 
tqdm.pandas()


""" 
path_list (블로그/유튜브/인스타/트위터) + 보고서
[0~3] - 2020/2021  0:블로그 / 1: 유튜브 / 2: 인스타 / 3: 트위터
[4~7] - 2022       4:블로그 / 5: 유튜브 / 6: 인스타 / 7: 트위터
[8]   - 보고서      8: 보고서

"""

name = set_name(path)
df = pd.DataFrame()
# df = file_process(4)

df_blog = file_process(4)
df_yt = file_process(5)
df_insta = file_process(6)
df_twitter = file_process(7)

df = pd.concat([df, df_blog, df_yt, df_insta, df_twitter], ignore_index=True)
# df.to_excel('소셜미디어_전처리.xlsx', encoding='utf-8-sig')

Tokenization Progress: 100%|████████████████████████████████████████████████████| 4357/4357 [00:00<00:00, 99356.18it/s]


In [111]:
df = pd.read_excel('소셜미디어_전처리.xlsx')

In [ ]:
df

In [ ]:
# 단어사전 확인
with open("C:\\mecab\\user-dic\\nnp.csv", 'r', encoding='utf-8') as f: 
    file_data = f.readlines()
file_data

In [ ]:
# 우선순위 확인
with open("c:/mecab/mecab-ko-dic/user-nnp.csv", 'r', encoding='utf-8') as f: 
    file_data = f.readlines()
file_data

In [108]:
df['tokens'].isna().sum()

0

### 5.JST

In [152]:
from jointtsmodel.RJST import RJST
from jointtsmodel.JST import JST
from jointtsmodel.sLDA import sLDA
from jointtsmodel.TSM import TSM
from jointtsmodel.TSWE import TSWE

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from jointtsmodel.utils import *


text = df['tokens'].apply(lambda x: ' '.join(x))
vectorizer = CountVectorizer(max_df=0.5,
                             min_df=5,
                             ngram_range=(1,2),          #바이그램 사용시 성능 확인
                             
                             max_features=3000)

X = vectorizer.fit_transform(text)
vocabulary = vectorizer.get_feature_names_out()

inv_vocabulary = dict(zip(vocabulary,np.arange(len(vocabulary))))
lexicon_data = pd.read_excel('./prior_sentiment_kor.xlsx')
lexicon_data = lexicon_data.dropna()
lexicon_dict = dict(zip(lexicon_data['Word'],lexicon_data['Sentiment']))

In [142]:
vocabulary[:100]

array(['가게', '가격', '가격 상승', '가계', '가공', '가구', '가동', '가뭄', '가방', '가볍',
       '가상', '가속', '가수', '가스', '가슴', '가시', '가요', '가운데', '가을', '가이드',
       '가이드라인', '가입', '가전', '가정', '가져가', '가족', '가죽', '가치', '가치 가치',
       '가치 동시', '가치 사회', '가치 소비', '가치 실현', '가치 창출', '가치 평가', '가치관', '각각',
       '각국', '각자', '각종', '간담회', '간식', '간접', '갈등', '감각', '감독', '감동', '감성',
       '감소', '감시', '감안', '감정', '감축', '감축 목표', '강남', '강남구', '강릉', '강사',
       '강산', '강세', '강아지', '강연', '강원', '강원도', '강의', '강점', '강제', '강조', '강하',
       '강화', '갖추', '개강', '개념', '개미', '개발', '개발 사업', '개발 협력', '개발자', '개방',
       '개별', '개사', '개선', '개설', '개요', '개인', '개인 정보', '개입', '개정', '개척',
       '개최', '개편', '개혁', '갤러리', '갤럭시', '거대', '거래', '거래소', '거버넌스', '거부',
       '거절'], dtype=object)

In [153]:

# for n in range(3, 16):

model = JST(n_topic_components=3, n_sentiment_components=3,random_state=123,evaluate_every=10)
model.fit(X.toarray(), lexicon_dict)

# model.transform()[:2]

top_words = list(model.getTopKWords(vocabulary).values())

'''
utils.py // def coherence_score_umass() 깃허브 확인 후 코드 수정 (127~129) 2023-03-19

'''
coherence_score_umass(X.toarray(),inv_vocabulary,top_words)           

Perplexity after iteration 10 (out of 10 iterations) is 3668.68


-1.0008362915306384

In [154]:
model.getTopKWords(vocabulary, num_words=10)   

{(1, 1): ['활동', '사회', '환경', '진행', '경영', '친환경', '참여', '활용', '실천', '제품'],
 (1, 2): ['사회', '경영', '활동', '교육', '지원', '환경', '진행', '가치', '참여', '사업'],
 (1, 3): ['친환경', '환경', '제품', '경영', '사용', '활용', '브랜드', '사업', '사회', '기술'],
 (2, 1): ['면접', '사업', '지원', '경영', '산업', '기술', '투자', '에너지', '시장', '안전'],
 (2, 2): ['사용', '환경', '시작', '제품', '친환경', '세대', '경영', '사랑', '브랜드', '선물'],
 (2, 3): ['투자', '경제', '시장', '가격', '세계', '에너지', '상승', '산업', '경영', '환경'],
 (3, 1): ['환경', '경영', '사회', '사용', '제품', '친환경', '실천', '인증', '지구', '여행'],
 (3, 2): ['경영', '환경', '사회', '면접', '평가', '탄소', '배출', '지원', '사업', '금융'],
 (3, 3): ['사회', '가치', '경제', '경영', '환경', '기술', '디지털', '투자', '챌린지', '금융']}

In [ ]:
# 수정사항 확인용
import inspect
print(inspect.getsource(coherence_score_umass))

In [12]:
model.perplexity()

1651.2570725141406

In [185]:
top_words = list(model.getTopKWords(vocabulary).values())